# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

S. Souza  ->  S. Souza  |  ['S. Souza']
S. Kraus  ->  S. Kraus  |  ['S. Kraus']
K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']


C. Gieser  ->  C. Gieser  |  ['C. Gieser']
T. Henning  ->  T. Henning  |  ['T. Henning']
D. Semenov  ->  D. Semenov  |  ['D. Semenov']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']


Arxiv has 92 new papers today
          8 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/8 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2507.15921


extracting tarball to tmp_2507.15921... done.


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2507.15921/aanda.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'isoc_fit' from 'tmp_2507.15921/isoc_fit.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'r_values' from 'tmp_2507.15921/r_values.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2

Found 51 bibliographic references in tmp_2507.15921/aanda.bbl.
Retrieving document from  https://arxiv.org/e-print/2507.15933


...

extracting tarball to tmp_2507.16303...

 done.


C. Gieser  ->  C. Gieser  |  ['C. Gieser']
T. Henning  ->  T. Henning  |  ['T. Henning']
D. Semenov  ->  D. Semenov  |  ['D. Semenov']


Found 80 bibliographic references in tmp_2507.16303/L1448N_PRODIGE_paper2_v7.bbl.
Retrieving document from  https://arxiv.org/e-print/2507.16317
extracting tarball to tmp_2507.16317...

 done.
Retrieving document from  https://arxiv.org/e-print/2507.16589
extracting tarball to tmp_2507.16589... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.15921-b31b1b.svg)](https://arxiv.org/abs/2507.15921) | **The VISCACHA Survey: XIII. The extended main-sequence turn-off in intermediate-age low-mass clusters**  |
|| <mark>S. Souza</mark>, et al. |
|*Appeared on*| *2025-07-23*|
|*Comments*| *Accepted for publication in A&A. 9 pages, 5 figures, and 2 tables*|
|**Abstract**|            The extended main-sequence turn-off (eMSTO) is a well-known feature observed in young and intermediate-age star clusters, characterized by a significant broadening of the main-sequence turn-off region. Although prolonged star formation and stellar rotation have been proposed as possible explanations, no consensus has yet been reached. Most previous studies have focused on high-mass clusters. In this work, we extend the analysis to the less-explored low-mass regime by investigating star clusters in the Magellanic Clouds using data from the VISCACHA survey. We employed a widely used method to quantify the MSTO width in terms of age spread. Additionally, to validate our approach, we used a cluster also observed with HST. Our analysis confirms that the eMSTO phenomenon is also present in low-mass clusters, following the known age/mass-MSTO width relations. In particular, the less massive cluster in our sample does not show an eMSTO, supporting the proposed link between the eMSTO and the escape velocity of the cluster, providing a new lower limit to the age spread of $88\pm40$ Myr for the presence of the eMSTO. The consistent MSTO width measurements between the VISCACHA and HST photometries confirm the robustness of our method and demonstrate that the age spread determination is independent of the photometric system, showing also the power of ground-based observations to investigate the eMSTO phenomenon.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.16303-b31b1b.svg)](https://arxiv.org/abs/2507.16303) | **ANTIHEROES-PRODIGE: Quantifying the connection from envelope to disk with the IRAM 30m telescope and NOEMA I. Attack of the streamers: L1448N's fight for order in the chaos**  |
|| <mark>C. Gieser</mark>, et al. -- incl., <mark>T. Henning</mark>, <mark>D. Semenov</mark> |
|*Appeared on*| *2025-07-23*|
|*Comments*| *accepted for publication in A&A, 13 pages, 9 figures*|
|**Abstract**|            Star formation is a hierarchical process ranging from molecular clouds down to individual protostars. In particular how infalling asymmetric structures, called streamers, delivering new material onto protostellar systems, are connected to the surrounding envelope is not understood. We investigate the connection between the cloud material at 10000 au scales down to 300 au scales towards L1448N in the Perseus star-forming region hosting three young Class 0/I protostellar systems. Sensitive molecular line observations taken with the IRAM 30m telescope and NOEMA at 1.4 mm are used to study the kinematic properties in the region traced by the molecular lines. Several infalling streamers are associated with the protostellar systems, some of them traced by C18O and DCN, while one of them is bright in SO and SO2. The kinematic properties of the former streamers are consistent with the velocities observed at large envelope scales of 10000 au, while the latter case show different kinematics. The masses and infall rates of the streamers are 0.01 Msun and 0.01-0.1 Msun and 10^-6 Msun/yr and 5-18x10^-6 Msun/yr for IRS3A and IRS3C, respectively. The envelope mass in the L1448N region is ~16Msun, thus the mass of a single streamer is low compared to the envelope mass (<1%). However, compared to the estimated mass of the protostellar systems a single streamer could deliver up to 1% and 8-17% of mass towards IRS3A (M*~1.2 Msun) and IRS3C (M*~1 Msun), respectively. The rotational signatures of structures in L1448N are all connected - from the large-scale envelope, infalling streamers, down to the rotation of all three disks. Two of the three Class 0/I protostellar systems are still fed by this surrounding material, which can be associated to the remnant envelope. However, we also find a streamer that is bright in SO and SO2 towards IRS3C that could be connected to a nearby sulfur reservoir.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.15952-b31b1b.svg)](https://arxiv.org/abs/2507.15952) | **Carbon burning cannot explain puffy hypervelocity white dwarfs**  |
|| N. Yamaguchi, <mark>K. El-Badry</mark>, T. L. S. Wong, K. J. Shen |
|*Appeared on*| *2025-07-23*|
|*Comments*| *11 pages, 9 figures, submitted to PASP*|
|**Abstract**|            The dominant formation channel of Type Ia supernovae (SNe Ia) has been extensively studied for many years. Several hypervelocity white dwarfs (HVWDs) with space velocities of $\gtrsim 1000\,\mathrm{kms}^{-1}$ have recently been discovered. One possible origin of these stars is the dynamically-driven double-degenerate double-detonation (D6) scenario, in which an accreting sub-Chandrasekhar mass carbon-oxygen (CO) WD detonates as a SN Ia. In this scenario, the less massive WD may survive its companion's detonation and be ejected as a HVWD. Most of the observed HVWDs are hotter and puffier than normal WDs, perhaps due to their recent proximity to a SN. In this work, we test whether these properties can be explained by long-lived stable carbon (C) burning in the interiors of CO WD donors triggered by a SN shock. We model the long-term evolution of CO WDs following rapid energy injection using 1D models. We find that stable C burning can be ignited in CO WDs with masses of $0.95 - 1.10\,M_{\odot}$ if SN energy penetrates sufficiently deeply. The resulting born-again stars settle on the C-burning main sequence while they convert their interiors from C and O to Ne and Mg, where they have temperatures and radii similar to some of the observed HVWDs. However, the timescale over which C-burning WDs remain inflated is $\lesssim 10^5\,$yr, which is at least an order of magnitude shorter than the kinematic ages of observed hot HVWDs. We conclude that observed HVWDs are unlikely to be inflated by C burning. The stellar evolution of observed HVWDs remains an open problem.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.16131-b31b1b.svg)](https://arxiv.org/abs/2507.16131) | **Central Concentration and Escape of Ionizing Photons in Galaxies at the Epoch of Reionization**  |
|| C. Lyu, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-07-23*|
|*Comments*| *16 pages, 7+4 figures. Accepted by ApJL*|
|**Abstract**|            Compact, low-mass galaxies with strong nebular emission are considered promising candidates for efficient ionizing photon production and escape. We present a spatially resolved analysis of 189 galaxies at redshifts $z \sim 6.7-7.6$ in JADES GOODS-N and GOODS-S fields and selected via JWST/NIRCam F410M filter. By employing annular photometry and spectral energy distribution fitting across rest-frame UV to optical wavelengths, we investigate the internal structure of star formation, ionizing photon production and escape, as well as the resolved star formation histories within these galaxies. We find that these galaxies exhibit compact, centrally concentrated, and bursty star formation, especially in lower-mass systems ($\log(M_*/{\rm M_{\odot}}) <9.0$). The central regions of them display extreme [OIII]+H$\beta$ equivalent widths ($>$1000 Å), high ionizing photon production efficiencies ($\xi_{\text{ion}} \sim 10^{25.6}$ Hz erg$^{-1}$), steep UV slopes ($\sim -2.3$), and elevated escape fractions ($f_{\text{esc}} > 0.08$), with all these properties peaking in the inner regions. These findings reveal outside-in growth and rising star formation histories at $z\sim 7$, with the central regions of them playing a pivotal role in driving cosmic reionization.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.16189-b31b1b.svg)](https://arxiv.org/abs/2507.16189) | **An AI super-resolution field emulator for cosmological hydrodynamics: the Lyman-α forest**  |
|| F. Hafezianzadeh, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-07-23*|
|*Comments*| **|
|**Abstract**|            We extend our super-resolution and emulation framework for cosmological dark matter simulations to include hydrodynamics. We present a two-stage deep learning model to emulate high-resolution (HR-HydroSim) baryonic fields from low-resolution (LR-HydroSim) simulations at redshift $z = 3$. The method takes as inputs an LR-HydroSim and the high-resolution initial conditions (HR-HydroICs). First, the model stochastically generates high-resolution baryonic fields from the LR-HydroSim. Second, a deterministic emulator refines these fields using HR-HydroICs to reconstruct small-scale structures including displacement, velocity, internal energy, and gas/star classification. Trained on paired low- and high-resolution simulations produced with \texttt{MP-Gadget}, the model captures small-scale structures of the intergalactic medium and %Lyman-$\alpha$ forest observables down to the 100 kpc pressure smoothing scale relevant to the Lyman-$\alpha$ forest. The model achieves subpercent error for overdensity, temperature, velocity, and optical depth fields, a mean relative error of 1.07\% in the large-scale flux power spectrum (\(k < 3 \times 10^{-2}\ \mathrm{s/km}\)), and less than 10\% error in the flux probability distribution function. Notably, the two-stage model reduces the compute time by a factor of $\sim$450 compared to full smoothed particle hydrodynamics at the same resolution. This work demonstrates the potential of this framework as a powerful and efficient tool for generating high-resolution fields offering fast and accurate alternatives to traditional cosmological hydrodynamic simulations and enabling large-volume mock datasets for next-generation cosmological surveys.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.16317-b31b1b.svg)](https://arxiv.org/abs/2507.16317) | **Lunar Orbital VLBI Experiment: motivation, scientific purposes and status**  |
|| X. Hong, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-07-23*|
|*Comments*| *Accepted for publication in Science China: Physics, Mechanics & Astronomy*|
|**Abstract**|            The Lunar Orbital VLBI Experiment (LOVEX) is a scientific component of the Chinese Lunar Exploration Project (CLEP) Chang'E-7. The spaceborne component of LOVEX is implemented onboard the relay satellite QueQiao-2, which was launched on 2024 March 20, and later placed into an elliptical selenocentric orbit. The LOVEX-specific payload consists of an X-band cryogenic receiver, a hydrogen maser frequency standard, and VLBI data formatting and acquisition electronics. Several components of the QueQiao-2 nominal onboard instrumentation, such as the 4.2-meter antenna, the data storage device, and the downlink communication system, contribute to the overall spaceborne VLBI instrumentation. This allows us to form a space radio telescope capable of co-observing with Earth-based radio telescopes in VLBI mode. In this space VLBI system, the length of the baseline extends up to approximately 380,000 km. This paper presents the LOVEX scientific objectives, architecture, instrumentation, pre-launch tests, in-flight verification and calibration, and the first in-flight detections of interferometric response (''fringes'') achieved through observations of the quasar AO 0235+164 and the Chang'E-6 orbital module, positioned at the Sun-Earth Lagrange point L2. These initial results demonstrate the successful performance of LOVEX, verifying its capability for both astronomical and spacecraft tracking observations at ultra-long VLBI baselines.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.16589-b31b1b.svg)](https://arxiv.org/abs/2507.16589) | **Cosmological Preference for a Positive Neutrino Mass at 2.7$σ$: A Joint Analysis of DESI DR2, DESY5, and DESY1 Data**  |
|| G.-H. Du, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-07-23*|
|*Comments*| *9 pages, 4 figures*|
|**Abstract**|            Neutrinos and dark energy (DE) have entered a new era of investigation, as the latest DESI baryon acoustic oscillation measurements tighten the constraints on the neutrino mass and suggest that DE may be dynamical rather than a cosmological constant. In this work, we investigate the cosmological implications of simultaneously incorporating $\sum m_\nu$ and $N_{\rm eff}$ within the framework of dynamical DE. A joint analysis of DESI DR2, cosmic microwave background, DESY5 supernova, and DESY1 weak lensing data yields a total neutrino mass of $\sum m_\nu = 0.098^{+0.016}_{-0.037}\,\mathrm{eV}$, indicating a preference for a positive neutrino mass at the $2.7\sigma$ level within the $w_0w_a$CDM framework. This work highlights the important influence of the strength of evidence for dynamically evolving DE on the measurement of neutrino mass, with stronger evidence for DE dynamics leading to a larger neutrino mass. These results underscore the importance of measuring the neutrino mass within the framework of dynamical DE, particularly in light of the forthcoming, more complete DESI data releases.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2507.15933-b31b1b.svg)](https://arxiv.org/abs/2507.15933) | **Orbital and Physical Properties of the Pleiades Binary 27 Tau (Atlas)**  |
|| G. Torres, et al. -- incl., <mark>S. Kraus</mark> |
|*Appeared on*| *2025-07-23*|
|*Comments*| *18 pages in emulateapj format, including figures and tables. Accepted for publication in The Astrophysical Journal*|
|**Abstract**|            We report new spectroscopic and interferometric observations of the Pleiades binary star Atlas, which played an important role nearly three decades ago in settling the debate over the distance to the cluster from ground-based and space-based determinations. We use the new measurements, together with other published and archival astrometric observations, to improve the determination of the 291-day orbit and the distance to Atlas ($136.2 \pm 1.4$ pc). We also derive the main properties of the components, including their absolute masses ($5.04 \pm 0.17 M_{\odot}$ and $3.64 \pm 0.12 M_{\odot}$), sizes, effective temperatures, projected rotational velocities, and chemical composition. We find that the more evolved primary star is rotationally distorted, and are able to estimate its oblateness and the approximate orientation of its spin axis from the interferometric observations. The spin axis may well be aligned with the orbital axis. Models of stellar evolution from MESA that account for rotation provide a good match to all of the primary's global properties, and point to an initial angular rotation rate on the zero-age main sequence of about 55% of the breakup velocity. The current location of the star in the H-R diagram is near the very end of the hydrogen-burning main sequence, at an age of about 105 Myr, according to these models. Our spectroscopic analysis of the more slowly-rotating secondary indicates that it is a helium-weak star, with other chemical anomalies.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2507.15921/./main_fig.png', 'tmp_2507.15921/./example_result_2.png', 'tmp_2507.15921/./ngc152_hst.png']
copying  tmp_2507.15921/./main_fig.png to _build/html/
copying  tmp_2507.15921/./example_result_2.png to _build/html/
copying  tmp_2507.15921/./ngc152_hst.png to _build/html/
exported in  _build/html/2507.15921.md
    + _build/html/tmp_2507.15921/./main_fig.png
    + _build/html/tmp_2507.15921/./example_result_2.png
    + _build/html/tmp_2507.15921/./ngc152_hst.png
found figures ['tmp_2507.16303/./moment0_L1448N_C18O_2_1_both.png', 'tmp_2507.16303/./moment0_L1448N_SO_6_5_5_4_both.png', 'tmp_2507.16303/./moment0_L1448N_SO2_4_2_2_3_1_2_both.png', 'tmp_2507.16303/./Continuum_L1448N_both.png', 'tmp_2507.16303/./multifit_L1448N_C18O_2_1_30m_0.png', 'tmp_2507.16303/./multifit_L1448N_C18O_2_1_merged_0.png', 'tmp_2507.16303/./multifit_L1448N_SO2_4_2_2_3_1_2_merged_0.png']
copying  tmp_2507.16303/./moment0_L1448N_C18O_2_1_both.png to _build/html/
copying  tmp_2507.16303/./

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\angstrom}{\mbox{\normalfontÅ}}$
$\newcommand{\orcid}[1]{\href{https://orcid.org/#1}{\includegraphics[width=10pt]{Orcid-ID.png}}}$
$\newcommand{\stefano}[1]{{\color{red}#1}}$</div>



<div id="title">

# The VISCACHA survey: XIII. The extended main-sequence turn-off in intermediate-age low-mass clusters

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.15921-b31b1b.svg)](https://arxiv.org/abs/2507.15921)<mark>Appeared on: 2025-07-23</mark> -  _Accepted for publication in A&A. 9 pages, 5 figures, and 2 tables_

</div>
<div id="authors">

S. O. Souza, et al.

</div>
<div id="abstract">

**Abstract:** The extended main-sequence turn-off (eMSTO) is a well-known feature observed in young and intermediate-age star clusters, characterized by a significant broadening of the main-sequence turn-off region. Although prolonged star formation and stellar rotation have been proposed as possible explanations, no consensus has yet been reached. Most previous studies have focused on high-mass clusters. In this work, we extend the analysis to the less-explored low-mass regime by investigating star clusters in the Magellanic Clouds using data from the VISCACHA survey. We employed a widely used method to quantify the MSTO width in terms of age spread. Additionally, to validate our approach, we used a cluster also observed with HST. Our analysis confirms that the eMSTO phenomenon is also present in low-mass clusters, following the known ${age/mass-MSTO width relations}$ . In particular, the less massive cluster in our sample does not show an eMSTO, supporting the proposed link between the eMSTO and the escape velocity of the cluster, providing a new lower limit to the age spread of $88\pm40$ Myr for the presence of the eMSTO. The consistent MSTO width measurements between the VISCACHA and HST photometries confirm the robustness of our method and demonstrate that the age spread determination is independent of the photometric system, showing also the power of ground-based observations to investigate the eMSTO phenomenon.

</div>

<div id="div_fig1">

<img src="tmp_2507.15921/./main_fig.png" alt="Fig4" width="100%"/>

**Figure 4. -** The MSTO width at 50\% of the maximum (FWHM), as a function of cluster mass (panel a), escape velocity (panel b), and age (panels c and d). The colored dots represent the star clusters analyzed in this work. Yellow squares represent MCs clusters from [Goudfrooij, Girardi and Kozhurina-Platais (2014)](), the blue squares are the clusters analyzed by [Correnti, Goudfrooij and Kalirai (2014)]() and used by [Goudfrooij, Girardi and Kozhurina-Platais (2014)]() lower limit for the existence of the eMSTO, yellow crosses indicate YMC analyzed by [Niederhofer, et. al (2015)](). Yellow triangles show MWOC from [Cordoni, Milone and Marino (2018)](). The yellow stars are the results by [ and Piatti (2016)](), [ and Piatti (2016)](). The colored dotted lines illustrate the expected MSTO width comparing non-rotating isochrones to models for different rotation fractions $\Omega/\Omega_C=$ 0.3 (blue), 0.6 (orange), 0.8 (green), and 0.99 (red), adopting the metallicity of [M/H]$=-0.6$(mean of our sample). The horizontal dashed blue line represents previous limits by [Correnti, Goudfrooij and Kalirai (2014)]() and [Goudfrooij, Girardi and Kozhurina-Platais (2014)](), while the pink line and shaded region mark the eMSTO limit derived in this work. The black lines in panels (a) and (b) represent the linear regression for the data from \citet[][dotted]{cordoni18}, \citet[][dashed]{goudfrooij14}, and this work (solid). The gray shaded regions represent the standard deviation of the fit.  (*fig:main_fig*)

</div>
<div id="div_fig2">

<img src="tmp_2507.15921/./example_result_2.png" alt="Fig3" width="100%"/>

**Figure 3. -** Hess diagrams for SL36 (left) and NGC152 (right) showing the MSTO width derivation. Panels a1 and b1 show the observed data expanded via Monte Carlo simulation. The synthetic CMDs generated from the best isochrone fitting are in panels a2 (SL36) and b2 (NGC152). The red arrow represents the reddening vector for a $E(B-V)=0.2$. The age distribution along the eMSTO are shown in panels a3 (SL36) and b3 (NGC152). The synthetic CMDs were generated assuming the scaled errors using the MS control sample and demonstrated in panels a4 (SL36) and b4 (NGC152). The dotted black lines in panels a1, a2, b1, and b2 are the support lines to construct the eMSTO parallelogram (red shaded region) and the MS control sample (blue shaded region). The vertical green line in panels a4 and b4 shows the best error scale, while the horizontal line is the mean age standard deviation in the MS control sample used as an error for the MSTO width. (*fig:sl36_res*)

</div>
<div id="div_fig3">

<img src="tmp_2507.15921/./ngc152_hst.png" alt="Fig2" width="100%"/>

**Figure 2. -** Same as Figure \ref{fig:sl36_res}, but for NGC 152 using HST data. (*fig:ngc152_hst*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.15921"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand{\arraystretch}{1.1}$
$\newcommand\as{\ifmmode{\rlap.} " \! \else {\rlap.} " \!\fi}$</div>



<div id="title">

# ANTIHEROES-PRODIGE: Quantifying the connection from envelope to disk with the IRAM 30m telescope and NOEMA: I. Attack of the streamers: L1448N's fight for order in the chaos

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2507.16303-b31b1b.svg)](https://arxiv.org/abs/2507.16303)<mark>Appeared on: 2025-07-23</mark> -  _accepted for publication in A&A, 13 pages, 9 figures_

</div>
<div id="authors">

<mark>C. Gieser</mark>, et al. -- incl., <mark>T. Henning</mark>, <mark>D. Semenov</mark>

</div>
<div id="abstract">

**Abstract:** Star formation is a hierarchical process ranging from molecular clouds down to individual protostars. In particular how infalling asymmetric structures, called streamers, delivering new material onto protostellar systems, are connected to the surrounding envelope is not understood. We aim to investigate the connection between the cloud material at 10 000 au scales down to 300 au scales towards L1448N in the Perseus star-forming region hosting three young Class 0/I protostellar systems (IRS3A, IRS3B, and IRS3C). Sensitive molecular line observations taken with the Institut de Radioastronomie Millimetrique (IRAM) 30m telescope and the Northern Extended Millimeter Array (NOEMA) at 1.4 mm are used to study the kinematic properties in the region traced by the molecular lines (C $^{18}$ O, SO, DCN, and SO $_{2}$ ). The temperature in the region is estimated using transitions of c-C $_{3}$ H $_{2}$ . Several infalling streamers are associated with the protostellar systems, some of them traced by C $^{18}$ O and DCN, while one of them is bright in SO and $SO_2$ . The kinematic properties of the former streamers are consistent with the velocities observed at large envelope scales of 10 000 au, while the latter case show different kinematics. The masses and infall rates of the streamers are 0.01 $M_\odot$ and 0.01-0.1 $M_\odot$ and 10 $^{-6}$ $M_\odot$ yr $^{-1}$ and $5-18\times10^{-6}$ $M_\odot$ yr $^{-1}$ for IRS3A and IRS3C, respectively. The envelope mass in the L1448N region is $\approx$ 16 $M_\odot$ , thus the mass of a single streamer is low compared to the envelope mass ( $<$ 1 \% ). However, compared to the estimated mass of the protostellar systems a single streamer could deliver up to 1 \% and 8-17 \% of mass towards IRS3A ( $M_* \approx 1.2$ $M_\odot$ ) and IRS3C ( $M_* \approx 1$ $M_\odot$ ), respectively. The rotational signatures of structures in L1448N are all connected - from the large-scale envelope, infalling streamers, down to the rotation of all three disks. Two of the three Class 0/I protostellar systems are still fed by this surrounding material, which can be associated to the remnant envelope. However, we also find a streamer that is bright in SO and SO $_{2}$ towards IRS3C that could be connected to a nearby sulfur reservoir. Further studies are required to study the diverse chemical compositions and the origin of the streamers.

</div>

<div id="div_fig1">

<img src="tmp_2507.16303/./moment0_L1448N_C18O_2_1_both.png" alt="Fig6.1" width="33%"/><img src="tmp_2507.16303/./moment0_L1448N_SO_6_5_5_4_both.png" alt="Fig6.2" width="33%"/><img src="tmp_2507.16303/./moment0_L1448N_SO2_4_2_2_3_1_2_both.png" alt="Fig6.3" width="33%"/>

**Figure 6. -** Line integrated intensity map of C$^{18}$O ($2-1$), SO ($6_{5}-5_{4}$), and SO$_{2}$($4_{2,2}-3_{1,3}$) of the IRAM 30m data (left) and merged or NOEMA data (right). In all panels the line integrated intensity is shown in color. Contours and arrows are the same as in Fig. \ref{fig:continuum}. The synthesized beam of the line and continuum data is shown in the bottom left and right corner, respectively. In all panels scale bars are shown in the top left corner. In the right panels, distinct structures (bridge and stick) seen in molecular emission are indicated by dashed ellipses. (*fig:C18OSOSO2mom0*)

</div>
<div id="div_fig2">

<img src="tmp_2507.16303/./Continuum_L1448N_both.png" alt="Fig5" width="100%"/>

**Figure 5. -** Continuum images toward L1448N. The left panel shows large-scale 1.1 mm continuum emission of L1448  ([Enoch, Young and Glenn 2006]()) . The beam size is shown in the bottom left. The map size of the IRAM 30m observations is highlighted by the grey dashed rectangle. The right panel shows the NOEMA 1.4 mm continuum of L1448N with contour levels at 5, 25, 60, 120, 200$\times \sigma_\mathrm{cont}$ with $\sigma_\mathrm{cont}$=0.78 mJy beam$^{-1}$. The triangles and white labels mark the positions of individual protostars  ([Tobin, Looney and Li 2016]())  and the pink cross marks the recently detected IRS3-ALMA source  ([Looney, Lin and Li 2025]()) . Bipolar outflow directions launched by the three protostellar systems are marked by red and blue arrows, including the blueshifted outflow lobe launched by L1448-mm towards the southeast. The synthesized beam of the NOEMA continuum data is shown in the bottom left corner. In both panels scale bars are shown in the top left corner. (*fig:continuum*)

</div>
<div id="div_fig3">

<img src="tmp_2507.16303/./multifit_L1448N_C18O_2_1_30m_0.png" alt="Fig8.1" width="33%"/><img src="tmp_2507.16303/./multifit_L1448N_C18O_2_1_merged_0.png" alt="Fig8.2" width="33%"/><img src="tmp_2507.16303/./multifit_L1448N_SO2_4_2_2_3_1_2_merged_0.png" alt="Fig8.3" width="33%"/>

**Figure 8. -** Main clusters extracted with \texttt{DBSCAN}. The velocity, peak intensity, and line width are shown in color in the top, central, and bottom panels, respectively. Contours and arrows are the NOEMA 1.4 mm continuum with the same levels as in Fig. \ref{fig:continuum}. In cluster 1 of each analyzed data set, the synthesized beam of the line and continuum data is shown in the bottom left and right corner, respectively, and scale bars are shown in the top left corner. The remaining clusters are shown in Fig. \ref{fig:clusters_app}. (*fig:clusters*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2507.16303"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

148  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

18  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

6  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
